# Telescopes - PCA Project

## Import Packages

In [1]:
import warnings  # Supress warnings

warnings.filterwarnings("ignore")
import os  # Interact with the operating system.
import sys  # Read system parameters.
from io import StringIO  # Read and write strings as files
from urllib.parse import urlparse  # Parse URL and extract path component

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests  # Send HTTP requests
import seaborn as sns
import sklearn
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

print("Libraries used in this project:")
print("- Python {}".format(sys.version))
print("- NumPy {}".format(np.__version__))
print("- pandas {}".format(pd.__version__))
print("- Matplotlib {}".format(mpl.__version__))
print("- Seaborn {}".format(sns.__version__))
print("- scikit-learn {}\n".format(sklearn.__version__))

Libraries used in this project:
- Python 3.9.20 (main, Oct  3 2024, 02:27:54) 
[Clang 14.0.6 ]
- NumPy 1.22.3
- pandas 1.4.1
- Matplotlib 3.8.4
- Seaborn 0.13.0
- scikit-learn 1.2.2



## Load the Dataset

In [2]:
"""Capture URL"""

url = "https://raw.githubusercontent.com/tyrantdavis/datasets/refs/heads/main/telescope.csv"

"""Get the request object"""
request = requests.get(url).text

"""Save as data frame and copy"""
telescopes = pd.read_csv(StringIO(request))
df = telescopes.copy()

"""Parse the URL"""
parsed_url = urlparse(url)
path = parsed_url.path
filename = os.path.basename(path)


print(f"Loaded {len(df)} telescope records from {filename}.")

Loaded 19020 telescope records from telescope.csv.
